### Go ahead, make my day. ###

So now, let's get the HTML of the AFI website above.

We'll start by importing the Beautiful Soup module to parse the HTML, urllib2 to connect to the internet, and pandas to save our data neatly.

In [10]:
from bs4 import BeautifulSoup
import urllib
import pandas as pd
import ssl

def get_HTML(url):
    ssl._create_default_https_context = ssl._create_unverified_context
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers = {'User-Agent':user_agent,} 
    req = urllib.request.Request(url, None, headers)
    response = urllib.request.urlopen(req)
    html = response.read() 
    soup = BeautifulSoup(html, 'html.parser')
    return soup
#soup = get_HTML("http://www.afi.com/100Years/quotes.aspx") 

<b>get_HTML</b> is a function which will make a request to our url (<b>req</b>), retrieve the html by reading all the text (<b>html</b>, and make it a BeautifulSoup instance to make it easy to parse the HTML for what we want.

<b>afi_soup</b> is a bunch of html text. 

A nice way to find our target HTML tags (that is, either "b" for bold text, "a" for website links, "table" for tables, etc.) is to go to the website on Chrome or Firefox, right-click and view the page source. Then CTRL+F to search for the table, (like search for "Frankly, my dear, I don't give a damn"). You will see the following:

In [15]:
soup = get_HTML("https://www.ecb.europa.eu/press/pressconf/2016/html/index.en.html") 

<dt>20/10/2016</dt>        
<dd>
<span class="doc-title">
<a href="/press/pressconf/2016/html/is161020.en.html">
Introductory statement to the press conference (with Q&amp;A)
</a>
</span>

In [22]:
dates = soup.find_all("dt")
print(len(dates))
print(dates)

8
[<dt>08/12/2016</dt>, <dt>20/10/2016</dt>, <dt>08/09/2016</dt>, <dt>21/07/2016</dt>, <dt>02/06/2016</dt>, <dt>21/04/2016</dt>, <dt>10/03/2016</dt>, <dt>21/01/2016</dt>]


In [18]:
links = soup.findAll("span", { "class" : "doc-title" })
print(len(links))

8


#### Oh no, many tables ####
Within our HTML page, we found 3 tables. So now we need to see if we should only analyze the first, second or third. We can do this by finding the first row of each table, and printing out the first 100 characters of the text.

So we can see that we need to use the 2nd table, <b>tables[1]</b>

In [13]:
quote_table = tables[1]

rows = quote_table.find_all("tr")

sample_row = rows[3]

Let's inspect some characteristics of a sample row, <b>sample_row</b>

In [15]:
print(sample_row)

<tr style="height:13.0pt">
<td sclass="lttext" valign="top">
<p><span style="font-size:9.0pt;font-family:Verdana; color:black"><b>3</b></span></p>
</td>
<td class="lttext" valign="top">
<p><span style="font-size:9.0pt;font-family:Verdana; color:black">You don't understand!  I coulda had class. I coulda been a contender. I could've been somebody, instead of a bum, which is what I am.</span></p>
</td>
<td class="lttext" valign="top">
<p><span style="font-size:9.0pt;font-family:Verdana; color:black"><a href="http://www.afi.com/members/catalog/DetailView.aspx?s=&amp;Movie=51286">ON THE WATERFRONT</a></span></p>
</td>
<td class="lttext" valign="top">
<p><span style="font-size:9.0pt;font-family:Verdana; color:black">1954</span></p>
</td>
</tr>


We see that it still contains a lot of tags. The meat of each row, as mentioned above, is in the "td" tag. So let's extract all the td tags of this row.

In [27]:
print(sample_row.find_all("td"))

[<td sclass="lttext" valign="top">\n<p><span style="font-size:9.0pt;font-family:Verdana; color:black"><b>3</b></span></p>\n</td>, <td class="lttext" valign="top">\n<p><span style="font-size:9.0pt;font-family:Verdana; color:black">You don't understand!  I coulda had class. I coulda been a contender. I could've been somebody, instead of a bum, which is what I am.</span></p>\n</td>, <td class="lttext" valign="top">\n<p><span style="font-size:9.0pt;font-family:Verdana; color:black"><a href="http://www.afi.com/members/catalog/DetailView.aspx?s=&amp;Movie=51286">ON THE WATERFRONT</a></span></p>\n</td>, <td class="lttext" valign="top">\n<p><span style="font-size:9.0pt;font-family:Verdana; color:black">1954</span></p>\n</td>]


We only want the text though. For example, within the row, and for one element we can print out the text by using the <i>text</i> method provided by BeautifulSoup:

In [28]:
print(sample_row.find_all("td")[0].text)
print(sample_row.find_all("td")[1].text)
print(sample_row.find_all("td")[2].text)
print(sample_row.find_all("td")[3].text)


3


You don't understand!  I coulda had class. I coulda been a contender. I could've been somebody, instead of a bum, which is what I am.


ON THE WATERFRONT


1954



Perfect! Looks like almost everything we want. It would be nice though to put this in a list of 4 elements: the quote number , the quote, the movie, and the year. I'll use the <b>map</b> function from Python to keep only the text of each "td" tag in the row.

In [30]:
row_elements = sample_row.find_all("td")
print(map(lambda x: x.text.strip(),row_elements))

[u'3', u"You don't understand!  I coulda had class. I coulda been a contender. I could've been somebody, instead of a bum, which is what I am.", u'ON THE WATERFRONT', u'1954']


We are now ready to cycle through all the rows, and then loop through all the elements of each row to extract only the quote number, quote, movie and year. I'll use <b>pandas</b> to store them neatly in a dataframe. Note, the first row of the table is just the column headers, which can be fed perfectly into the columns argument of the pandas dataframes.

In [31]:
first = True
for row in quote_table.find_all("tr"):
	row_elements = row.find_all('td')
	row_txt = map(lambda x: x.text.strip(),row_elements)
	if first:
		final_df = pd.DataFrame(columns=row_txt)
		first = False
		continue
	final_df.loc[len(final_df)+1] = row_txt

Let's check out our database:

In [32]:
print(final_df.head())

   #                                              QUOTE               MOVIE  \
1  1             Frankly, my dear, I don't give a damn.  GONE WITH THE WIND   
2  2       I'm gonna make him an offer he can't refuse.       THE GODFATHER   
3  3  You don't understand!  I coulda had class. I c...   ON THE WATERFRONT   
4  4  Toto, I've a feeling we're not in Kansas anymore.    THE WIZARD OF OZ   
5  5                        Here's looking at you, kid.          CASABLANCA   

   YEAR  
1  1939  
2  1972  
3  1954  
4  1939  
5  1942  


### More Dynamic Web-Scraping ###

Suppose we also wanted to get more information of each quote, but this required going to the actual link of the movie. For example, we want to see who directed Gone With the Wind (who is Victor Fleming, Alex) which we can see by clicking on the link to the movie. <a> http://www.afi.com/members/catalog/DetailView.aspx?s=&Movie=1181</a>. 

Note that this link shows up in the HTML of the row where Gone with the Wind is located

In [33]:
print(rows[1])

<tr style="height:13.0pt">
<td sclass="lttext" valign="top">
<p><span style="font-size:9.0pt;font-family:Verdana; color:black"><b>1</b></span></p>
</td>
<td class="lttext" valign="top">
<p><span style="font-size:9.0pt;font-family:Verdana; color:black">Frankly, my dear, I don't give a damn.</span></p>
</td>
<td class="lttext" valign="top">
<p><span style="font-size:9.0pt;font-family:Verdana; color:black"><a href="http://www.afi.com/members/catalog/DetailView.aspx?s=&amp;Movie=1181">GONE WITH THE WIND</a></span></p>
</td>
<td class="lttext" valign="top">
<p><span style="font-size:9.0pt;font-family:Verdana; color:black">1939</span></p>
</td>
</tr>


In [36]:
print(rows[1].find_all("td")[2])

<td class="lttext" valign="top">
<p><span style="font-size:9.0pt;font-family:Verdana; color:black"><a href="http://www.afi.com/members/catalog/DetailView.aspx?s=&amp;Movie=1181">GONE WITH THE WIND</a></span></p>
</td>


We can access specific attributes within an HTML tag by calling it as if it were a dictionary.

In [38]:
rows[1].find_all("td")[2].find("a")["href"]

u'http://www.afi.com/members/catalog/DetailView.aspx?s=&Movie=1181'

Thus, to find the link we just need to find the "a" element of the cell and call "href". 

Next, looking at the Gone with the Wind page source, the word <b>Director:</b> is in bold, followed by Victor Flemming. Fortunately, we can use BeautifulSoup to find the tag with <b>Director:</b> and then retrieve the next element by simply using the <b>find_next()</b> method of BeautifulSoup. 

We will do this in two steps.
    1. Run <b>get_HTML</b> on the link to the movie which we retrieve with .find("a")["href"]
    2. Find <b>Director:</b>, then get the next element which will presumably be the name.
    
### NOTE:  I will only do this for the first 10 movies on the link. Data scraping can cause a lot of traffic to the site and they can tell you are a web-scraper by making hundreds of requests rapidly from one IP. 

In [39]:
final_df['Director'] = ""

max_iter = 10
index = 0
for row in quote_table.find_all("tr"):
	if index>max_iter:
		break
	if row.find("a"):
		movie_soup = get_HTML(row.find("a")['href'])
		director = movie_soup.find(text="Director:").find_next().text
		final_df.loc[index,"Director"] = director
	index+=1

In [40]:
print(final_df.head())

   #                                              QUOTE               MOVIE  \
1  1             Frankly, my dear, I don't give a damn.  GONE WITH THE WIND   
2  2       I'm gonna make him an offer he can't refuse.       THE GODFATHER   
3  3  You don't understand!  I coulda had class. I c...   ON THE WATERFRONT   
4  4  Toto, I've a feeling we're not in Kansas anymore.    THE WIZARD OF OZ   
5  5                        Here's looking at you, kid.          CASABLANCA   

   YEAR                     Director  
1  1939          Victor Fleming       
2  1972    Francis Ford Coppola       
3  1954              Elia Kazan       
4  1939          Victor Fleming       
5  1942          Michael Curtiz       


Lastly, we can save our database nicely with Pandas as a Stata, Excel, CSV or R file for text analysis

In [41]:
final_df.to_csv("movie_quotes.csv",index=False, encoding='utf-8')

If you want more information, check out the BeautifulSoup docs <a>https://www.crummy.com/software/BeautifulSoup/bs4/doc</a>.

If you need to datascrape java websites or non-html sites, I recommend Selenium <a>selenium-python.readthedocs.io/</a>.

<a id='codecell'></a>

In [43]:
#################################
### Author: Paul Soto 		  ###
### 		paul.soto@upf.edu ###
#								#
# This file shows the basic of ##
# BeautifulSoup to datascrape ###
# an HTML based website and save#
# the data as a csv. 			#
#################################

from bs4 import BeautifulSoup
import urllib2
import pandas as pd

def get_HTML(url):
	"""
	This file creates a HTML soup from a given url
	"""
	req = urllib2.Request(url, headers={ 'User-Agent': 'Mozilla/5.0' })
	html = urllib2.urlopen(req).read()
	soup = BeautifulSoup(html, 'html.parser')
	return soup


# Get HTML Soup
afi_soup = get_HTML("http://www.afi.com/100Years/quotes.aspx")

# Isolate the table with the  
tables = afi_soup.find_all("table")
quote_table = tables[1]

# Loop through each row, retrieving the 
# quote ID number, quote, movie and year
first = True
for row in quote_table.find_all("tr"):
	# Get row elements
	row_elements = row.find_all('td')
	# Strip unnecessary text
	row_txt = map(lambda x: x.text.strip(),row_elements)
	if first:
		final_df = pd.DataFrame(columns=row_txt)
		first = False
		continue
	# Add to dataset
	final_df.loc[len(final_df)+1] = row_txt

# Clicking movie link to get the director
final_df['Director'] = ""
max_iter = 10
index = 0
for row in quote_table.find_all("tr"):
	if index>max_iter:
		break
	if row.find("a"):
		movie_soup = get_HTML(row.find("a")['href'])
		director = movie_soup.find(text="Director:").find_next().text
		final_df.loc[index,"Director"] = director
	index+=1

# Export to CSV
final_df.to_csv("movie_quotes.csv",index=False)